In [7]:
import pandas as pd
import numpy as np

In [8]:
def train_test_split(X, y, test_size=0.2, random_state=None):
    """
    Split the data into training and testing sets.

    Parameters:
    - X (array-like): The input features.
    - y (array-like): The target variable.
    - test_size (float, optional): The proportion of the data to include in the test set. Default is 0.2.
    - random_state (int, optional): The seed value for the random number generator. Default is None.

    Returns:
    - X_train (array-like): The training set input features.
    - X_test (array-like): The test set input features.
    - y_train (array-like): The training set target variable.
    - y_test (array-like): The test set target variable.
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    # Shuffle the indices
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    # Calculate the number of samples for the test set
    test_samples = int(len(X) * test_size)
    
    # Split the indices into train and test sets
    test_indices = indices[:test_samples]
    train_indices = indices[test_samples:]
    
    # Split the data based on the indices and return them
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]


In [9]:
def predict_from_model(training, target, testing):
    """
    Predicts the values of the target variable using an affine model fitted to the training data.

    Args:
        training (numpy.ndarray): Array of shape (n, m) containing the training data.
        target (numpy.ndarray): Array of shape (n, 1) containing the target variable.
        testing (numpy.ndarray): Array of shape (k, m) containing the testing data.

    Returns:
        numpy.ndarray: Array of shape (k, 1) containing the predicted values of the target variable.
    """
    # Add a column of ones to the training data
    training = np.column_stack((training, np.ones(len(training))))
    testing = np.column_stack((testing, np.ones(len(testing))))

    # Compute the least squares solution
    theta, _, _, _ = np.linalg.lstsq(training, target)

    # Predict the values of the target variable
    return testing @ theta


In [10]:
def moore(file_path: str) -> float:
    """
    Calculates the doubling time (in years) for transistor counts estimated via least squares
    for the data points in the given CSV file.

    Args:
        file_path (str): The path of the CSV file containing the data.

    Returns:
        float: The doubling time (in years) for transistor counts.

    """
    # Read the data from the CSV file
    data = pd.read_csv(file_path)

    # Extract the data from the DataFrame
    A = data['Year'].to_numpy()
    
    b = data['Transistors'].to_numpy()
    # Compute the base 2 logarithm of the transistor counts
    b = np.log2(b)

    # Fit an affine model to the data
    training = np.column_stack((A, np.ones(len(A))))
    gamma, _, _, _ = np.linalg.lstsq(training, b)

    return 1/gamma[0]

In [11]:
def min_validation_error(features, b) -> int:
    """
    Finds the value of k that minimizes the sum of squared residuals on the validation set.

    Args:
        features (numpy array): Array of shape (n, m) containing the dataset features.
        b (numpy array): Array of shape (n,) containing the target values.

    Returns:
        int: The value of k that minimizes the sum of squared residuals on the validation set.
    """

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features, b, test_size=0.5, random_state=42)

    min_error = 0
    best_k = 0
    for k in range(features.shape[1]):
        selected_features = X_train[:, :k+1]

        # Compute the least squares solution for the training set
        theta, _, _, _ = np.linalg.lstsq(selected_features, y_train)

        # Compute the sum of squared residuals on the validation set
        val_features = X_val[:, :(k+1)]
        val_predictions = val_features @ theta # use the model to predict CMEDV values for the validation set
        residuals = val_predictions - y_val
        error = (residuals**2).sum()

        if k == 1 or error < min_error:
            min_error = error
            best_k = k
    
    return best_k

In [12]:

data = pd.read_csv('transistors.csv')

A = data['Year'].to_numpy()
b = data['Transistors'].to_numpy()

# Assuming 'data' is your dataset
X_train, X_test, y_train, y_test = train_test_split(A, b, test_size=0.2, random_state=42)


predict_from_model(X_train, y_train, X_test)

C:\Users\1samk\AppData\Local\Temp\ipykernel_18780\520671929.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, _, _, _ = np.linalg.lstsq(training, target)


array([ 5.07881158e+09,  2.16712289e+09,  5.26079212e+09,  1.62118126e+09,
        1.25722018e+09, -2.20041014e+09, -2.38239068e+09, -2.01842960e+09,
        4.35088941e+09,  4.35088941e+09,  4.71485049e+09,  5.26079212e+09,
       -1.83644905e+09,  5.26079212e+09,  3.98692832e+09,  5.62475321e+09,
        5.26079212e+09,  1.62118126e+09,  5.26079212e+09, -1.66436248e+07,
       -5.62585254e+08, -1.29050743e+09,  1.80316180e+09,  3.44098669e+09,
        1.98514235e+09,  3.98692832e+09,  3.44098669e+09, -1.65446851e+09,
        1.65336918e+08,  2.16712289e+09,  5.26079212e+09,  1.98514235e+09,
       -2.56437123e+09,  4.53286995e+09,  3.07702561e+09,  5.26079212e+09,
        3.62296723e+09,  5.62475321e+09])